In [1]:
from setup import setup_src_path
print(setup_src_path())

['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


In [2]:
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload

print(config.Config.TXT_SAVE_PATH)
print(config.Config.MODEL_SAVE_PATH)


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-21 21:16:14.800697: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 21:16:14.836189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

./text-files/
./hp-model-


In [33]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [34]:
source_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 75013
})

In [7]:
from transformers import AutoConfig
from adapters import AutoAdapterModel

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
   
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [48]:
from torch.utils.data import DataLoader
eval_target_loader = DataLoader(test_target_data, batch_size=config.Config.BATCH_SIZE, shuffle=True)


In [46]:
adapter_loaded = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/task_adapter")

In [37]:
model.set_active_adapters(adapter_loaded)

In [38]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, eval_target_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7090080971659919
F1 score before adaptation: 0.7095148656707215


In [58]:
domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel")
task_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/stacked-adapter-task-modefied")

In [59]:
import adapters.composition as ac
model.active_adapters = ac.Stack(domain_adapter_coral, task_adapter)

In [60]:
model.active_adapters

Stack[domain_adapter_telephone-travel, stacked-adapter-task-modefied]

In [61]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, eval_target_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7105263157894737
F1 score before adaptation: 0.7116428701675159


In [4]:
domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel")
mlm_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/mlm-after-coral")
import adapters.composition as ac
model.active_adapters = ac.Stack(domain_adapter_coral, mlm_adapter)

In [8]:

from transformers import pipeline,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

mask_filler = pipeline("fill-mask", model=model, tokenizer=tokenizer)
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my appointment.
>>> i cancelled my flight.
>>> i cancelled my application.
>>> i cancelled my engagement.
>>> i cancelled my appointments.
